Ali mamaghani 401200306

In [ ]:
%pip install torchvision 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.3/760.3 KB 1.3 MB/s eta 0:00:001.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision.models import resnet18,resnet50, ResNet50_Weights
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import default_collate


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
model_1=resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
for param in model_1.parameters():
    param.requires_grad = False
model_1.fc = nn.Linear(2048, 10)


In [3]:
data_tr=datasets.CIFAR10(root='./cifar',
                           train=True,
                           download=True,
                           transform=transforms.Compose([
                                      transforms.Resize((224,224)),
                                      ToTensor()]
                          ))
data_te=datasets.CIFAR10(root='./cifar',
                           train=False,
                           download=True,
                           transform=transforms.Compose([
                                      transforms.Resize((224,224)),
                                      ToTensor()]
                          ))


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar/cifar-10-python.tar.gz to ./cifar
Files already downloaded and verified


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_dataloader = DataLoader(data_tr, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_dataloader = DataLoader(data_te, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [ ]:
model_1=model_1.to(device)

In [ ]:
learning_rate=0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_1.fc.parameters(), lr=learning_rate)

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_1, loss_fn, optimizer)
    test_loop(test_dataloader, model_1, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.290856  [    0/50000]
loss: 0.461566  [ 6400/50000]
loss: 0.515392  [12800/50000]
loss: 0.361939  [19200/50000]
loss: 0.401370  [25600/50000]
loss: 0.593746  [32000/50000]
loss: 0.637050  [38400/50000]
loss: 0.537752  [44800/50000]
Test Error: 
 Accuracy: 81.2%, Avg loss: 0.548218 

Done!


In [ ]:
torch.save(model_1, '/content/drive/MyDrive/Colab Notebooks/HW3/model_1.pth')

In [ ]:
model_1 = torch.load('/content/drive/MyDrive/model_1.pth')
model_1.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model_1 = torch.load('/content/drive/MyDrive/Colab Notebooks/HW3/model_1.pth')
model_1.eval()

In [ ]:
model_1 = torch.load('/content/drive/MyDrive/Colab Notebooks/HW3/model_1.pth',map_location=torch.device('cpu'))
model_1.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In this part we reached at 81% accuracy. It is not very high because we only traind last layer

# B

In [ ]:
model_2 = resnet18()
model_2.fc = nn.Linear(512, 10)

In [ ]:
for param in model_1.parameters():
    param.requires_grad = False

In [ ]:
def my_loss(zs,zt,y,t,alpha):
  def my_logit(z,t1): 
    #A = torch.exp(z/t)
    #y = A/torch.sum(A)
    return z/t1
  def my_ce(z1,z2):
    dd1,dd2=z1.size()
    z1max=torch.max(z1)
    z2max=torch.max(z1)
    A1 = torch.exp(z1-z1max+10**-5)
    y1 = A1/torch.sum(A1,axis=1).repeat(dd2,1).T
    A2 = torch.exp(z2-z2max+10**-5)
    y2 = A2/torch.sum(A2,axis=1).repeat(dd2,1).T
    #if torch.sum(torch.isnan(y1))>=1:
    #  print(y1,A1,"111")
    #if torch.sum(torch.isnan(y2))>=1:
    #  print(y2,A2,"222")
    return   -torch.sum(torch.log(y2+10**-6) * y1)/dd1
  
  celoss = nn.CrossEntropyLoss()
  floss = (1-alpha)*celoss(zs,y)+alpha*(t*t)*my_ce(my_logit(zt,t),my_logit(zs,t))
  return floss

By trial and error, we find that best alpha=0.5 and best t=1.2

In [ ]:
def train_loop(dataloader, model_s,model_t, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        z_s = model_s(X)
        z_t = model_t(X)
        
        loss = loss_fn(z_s,z_t,y,1.2,0.5)
        # Backpropagation
        if torch.sum(torch.isnan(loss))==0:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if batch % 100 == 0:
              loss, current = loss.item(), batch * len(X)
              print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_dataloader = DataLoader(data_tr, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_dataloader = DataLoader(data_te, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [ ]:
model_2=model_2.to(device)

In [ ]:
learning_rate=0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_2.parameters(), lr=learning_rate)

epochs = 80
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_2, model_1, my_loss, optimizer)
    test_loop(test_dataloader, model_2, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.883456  [    0/50000]
loss: 2.168281  [ 6400/50000]
loss: 1.969946  [12800/50000]
loss: 1.928281  [19200/50000]
loss: 1.628221  [25600/50000]
loss: 1.479555  [32000/50000]
loss: 1.286718  [38400/50000]
loss: 1.371918  [44800/50000]
Test Error: 
 Accuracy: 66.2%, Avg loss: 0.951243 

Epoch 2
-------------------------------
loss: 1.456535  [    0/50000]
loss: 1.426849  [ 6400/50000]
loss: 1.149502  [12800/50000]
loss: 1.301512  [19200/50000]
loss: 1.321746  [25600/50000]
loss: 1.188286  [32000/50000]
loss: 1.172272  [38400/50000]
loss: 1.129926  [44800/50000]
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.722655 

Epoch 3
-------------------------------
loss: 1.103823  [    0/50000]
loss: 1.131724  [ 6400/50000]
loss: 1.093106  [12800/50000]
loss: 1.098993  [19200/50000]
loss: 0.753218  [25600/50000]
loss: 0.968276  [32000/50000]
loss: 1.148276  [38400/50000]
loss: 0.900800  [44800/50000]
Test Error: 
 Accuracy: 79.2%, Avg loss: 0.605145 

Epoc

KeyboardInterrupt: ignored

In [ ]:
torch.save(model_2, '/content/drive/MyDrive/model_2.pth')

In [ ]:
torch.save(model_2, '/content/drive/MyDrive/Colab Notebooks/HW3/model_2.pth')

RuntimeError: ignored

In [ ]:
model_3 = torch.load('/content/drive/MyDrive/Colab Notebooks/HW3/model_3.pth')
model_3.eval()

In this part we reached at 87.2% accuracy.

# C

In [ ]:
model_3 = resnet18()
model_3.fc = nn.Linear(512, 10)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_dataloader = DataLoader(data_tr, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_dataloader = DataLoader(data_te, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [ ]:
model_3=model_3.to(device)

In [ ]:
learning_rate=0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_3.parameters(), lr=learning_rate)

epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_3, loss_fn, optimizer)
    test_loop(test_dataloader, model_3, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.081818  [    0/50000]
loss: 0.140075  [ 6400/50000]
loss: 0.108885  [12800/50000]
loss: 0.124799  [19200/50000]
loss: 0.277067  [25600/50000]
loss: 0.213719  [32000/50000]
loss: 0.198992  [38400/50000]
loss: 0.082656  [44800/50000]
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.597950 

Epoch 2
-------------------------------
loss: 0.020683  [    0/50000]
loss: 0.057636  [ 6400/50000]
loss: 0.049150  [12800/50000]
loss: 0.071741  [19200/50000]
loss: 0.141588  [25600/50000]
loss: 0.241969  [32000/50000]
loss: 0.122756  [38400/50000]
loss: 0.138439  [44800/50000]
Test Error: 
 Accuracy: 82.8%, Avg loss: 0.678082 

Done!


64.2-73.2-79.4-81.3-83.2-83.1-84.2-83.9-84.7-84-82.8

In [ ]:
torch.save(model_3, '/content/drive/MyDrive/Colab Notebooks/HW3/model_3.pth')

In [ ]:
model_3 = torch.load('/content/drive/MyDrive/model_3.pth')
model_3.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this part we reached at 84% accuracy. it is lower than previous part(87.2). It shows that model learns better when we have a teacher.

# D

In [ ]:
model_4=resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model_4.fc = nn.Linear(2048, 10)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_dataloader = DataLoader(data_tr, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_dataloader = DataLoader(data_te, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [ ]:
model_4=model_4.to(device)

In [ ]:
learning_rate=0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_4.parameters(), lr=learning_rate)

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    #train_loop(train_dataloader, model_4, loss_fn, optimizer)
    test_loop(test_dataloader, model_4, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 91.3%, Avg loss: 0.273944 

Done!


In [ ]:
torch.save(model_4, '/content/drive/MyDrive/Colab Notebooks/HW3/model_4.pth')

In [6]:
model_4 = torch.load('/content/drive/MyDrive/model_4.pth')
model_4.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model_5 = resnet18()
model_5.fc = nn.Linear(512, 10)

In [12]:
for param in model_4.parameters():
    param.requires_grad = False

In [11]:
def my_loss(zs,zt,y,t,alpha):
  def my_logit(z,t1): 
    #A = torch.exp(z/t)
    #y = A/torch.sum(A)
    return z/t1
  def my_ce(z1,z2):
    dd1,dd2=z1.size()
    z1max=torch.max(z1)
    z2max=torch.max(z1)
    A1 = torch.exp(z1-z1max+10**-5)
    y1 = A1/torch.sum(A1,axis=1).repeat(dd2,1).T
    A2 = torch.exp(z2-z2max+10**-5)
    y2 = A2/torch.sum(A2,axis=1).repeat(dd2,1).T
    #if torch.sum(torch.isnan(y1))>=1:
    #  print(y1,A1,"111")
    #if torch.sum(torch.isnan(y2))>=1:
    #  print(y2,A2,"222")
    return   -torch.sum(torch.log(y2+10**-6) * y1)/dd1
  
  celoss = nn.CrossEntropyLoss()
  floss = (1-alpha)*celoss(zs,y)+alpha*(t*t)*my_ce(my_logit(zt,t),my_logit(zs,t))
  return floss

In [8]:
def train_loop(dataloader, model_s,model_t, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        z_s = model_s(X)
        z_t = model_t(X)
        
        loss = loss_fn(z_s,z_t,y,1.2,0.5)
        # Backpropagation
        if torch.sum(torch.isnan(loss))==0:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if batch % 100 == 0:
              loss, current = loss.item(), batch * len(X)
              print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
train_dataloader = DataLoader(data_tr, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
test_dataloader = DataLoader(data_te, batch_size=64, shuffle=True,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [10]:
model_5=model_5.to(device)

In [ ]:
learning_rate=0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_5.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_5, model_4, my_loss, optimizer)
    test_loop(test_dataloader, model_5, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.861970  [    0/50000]
loss: 1.976185  [ 6400/50000]
loss: 1.962368  [12800/50000]
loss: 1.810928  [19200/50000]
loss: 1.522216  [25600/50000]
loss: 1.443438  [32000/50000]
loss: 1.168705  [38400/50000]
loss: 1.408040  [44800/50000]
Test Error: 
 Accuracy: 64.1%, Avg loss: 1.014717 

Epoch 2
-------------------------------
loss: 1.082161  [    0/50000]
loss: 1.085706  [ 6400/50000]
loss: 0.870673  [12800/50000]
loss: 1.073024  [19200/50000]
loss: 1.077772  [25600/50000]
loss: 1.106269  [32000/50000]
loss: 1.055206  [38400/50000]
loss: 0.808896  [44800/50000]
Test Error: 
 Accuracy: 75.5%, Avg loss: 0.697240 

Epoch 3
-------------------------------
loss: 0.815645  [    0/50000]
loss: 0.889142  [ 6400/50000]
loss: 0.756849  [12800/50000]
loss: 0.767936  [19200/50000]
loss: 0.736814  [25600/50000]
loss: 0.735108  [32000/50000]
loss: 0.631636  [38400/50000]
loss: 0.967609  [44800/50000]
Test Error: 
 Accuracy: 80.4%, Avg loss: 0.567948 

Epoc

In [15]:
learning_rate=0.0001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_5.parameters(), lr=learning_rate)

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_5, model_4, my_loss, optimizer)
    test_loop(test_dataloader, model_5, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.085818  [    0/50000]
loss: 0.180335  [ 6400/50000]
loss: 0.153948  [12800/50000]
loss: 0.164856  [19200/50000]
loss: 0.135125  [25600/50000]
loss: 0.148128  [32000/50000]
loss: 0.128852  [38400/50000]
loss: 0.177186  [44800/50000]
Test Error: 
 Accuracy: 89.3%, Avg loss: 0.325759 

Done!


In [ ]:
torch.save(model_5, '/content/drive/MyDrive/model_5.pth')

In [7]:
model_5 = torch.load('/content/drive/MyDrive/model_5.pth')
model_5.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In this part we reached at 91% accuracy for teacher. However, in part (a) we had 81% accuracy. It shows that when we learn all layers we reach at higher accuracy.</br>
Furthermore, for student model we reached at 89% accuracy, which was 87% in part (b). Therefore, we can conclude that the more our teacher works better, the more our student is accurate.
